# 02: Implementing BFS and DFS

## Learning Objectives
- Implement Breadth-First Search (BFS) from scratch
- Implement Depth-First Search (DFS) from scratch
- Understand the differences in exploration strategies
- Analyze time and space complexity
- Visualize search algorithm behavior

## 1. Review: Search Algorithm Components

In [ ]:
# Import required modules
import sys
sys.path.append('../src')

from typing import Any, List, Tuple, Set, Dict
from collections import deque
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import time

In [ ]:
# Import our base classes from notebook 01
class SearchProblem:
    def get_start_state(self) -> Any:
        raise NotImplementedError
    
    def is_goal_state(self, state: Any) -> bool:
        raise NotImplementedError
    
    def get_successors(self, state: Any) -> List[Tuple[Any, str, float]]:
        raise NotImplementedError

class GridSearchProblem(SearchProblem):
    def __init__(self, grid, start, goal):
        self.grid = grid
        self.start = start
        self.goal = goal
        self.rows = len(grid)
        self.cols = len(grid[0])
    
    def get_start_state(self):
        return self.start
    
    def is_goal_state(self, state):
        return state == self.goal
    
    def get_successors(self, state):
        successors = []
        row, col = state
        moves = [(-1, 0, 'UP'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (0, 1, 'RIGHT')]
        
        for dr, dc, action in moves:
            new_row, new_col = row + dr, col + dc
            if (0 <= new_row < self.rows and 0 <= new_col < self.cols and 
                self.grid[new_row][new_col] != 1):
                successors.append(((new_row, new_col), action, 1.0))
        return successors

## 2. Breadth-First Search (BFS)

BFS explores all nodes at depth d before exploring nodes at depth d+1.

### Key Properties:
- **Data Structure**: Queue (FIFO)
- **Complete**: Yes (finds solution if it exists)
- **Optimal**: Yes (for uniform costs)
- **Time**: O(b^d) where b=branching factor, d=depth
- **Space**: O(b^d)

In [ ]:
class BreadthFirstSearch:
    """BFS implementation with path tracking and statistics"""
    
    def __init__(self):
        self.nodes_expanded = 0
        self.max_frontier_size = 0
        self.exploration_order = []  # For visualization
    
    def search(self, problem: SearchProblem) -> List[str]:
        """
        Search for solution using BFS
        Returns: List of actions from start to goal
        """
        # Initialize
        start_state = problem.get_start_state()
        
        # Check if start is goal
        if problem.is_goal_state(start_state):
            return []
        
        # Frontier: queue of (state, path) tuples
        frontier = deque([(start_state, [])])
        
        # Explored set to avoid cycles
        explored = set()
        
        # Search loop
        while frontier:
            # Update statistics
            self.max_frontier_size = max(self.max_frontier_size, len(frontier))
            
            # Get next state (FIFO)
            state, path = frontier.popleft()
            
            # Skip if already explored
            if state in explored:
                continue
            
            # Mark as explored
            explored.add(state)
            self.nodes_expanded += 1
            self.exploration_order.append(state)
            
            # Expand node
            for successor, action, cost in problem.get_successors(state):
                if successor not in explored:
                    # Check if goal
                    if problem.is_goal_state(successor):
                        self.exploration_order.append(successor)
                        return path + [action]
                    
                    # Add to frontier
                    frontier.append((successor, path + [action]))
        
        # No solution found
        return None
    
    def get_statistics(self):
        """Return search statistics"""
        return {
            'nodes_expanded': self.nodes_expanded,
            'max_frontier_size': self.max_frontier_size,
            'states_explored': len(self.exploration_order)
        }

## 3. Depth-First Search (DFS)

DFS explores as deep as possible before backtracking.

### Key Properties:
- **Data Structure**: Stack (LIFO)
- **Complete**: No (can get stuck in infinite paths)
- **Optimal**: No
- **Time**: O(b^m) where m=maximum depth
- **Space**: O(bm)

In [ ]:
class DepthFirstSearch:
    """DFS implementation with path tracking and statistics"""
    
    def __init__(self):
        self.nodes_expanded = 0
        self.max_frontier_size = 0
        self.exploration_order = []
    
    def search(self, problem: SearchProblem) -> List[str]:
        """
        Search for solution using DFS
        Returns: List of actions from start to goal
        """
        # Initialize
        start_state = problem.get_start_state()
        
        # Check if start is goal
        if problem.is_goal_state(start_state):
            return []
        
        # Frontier: stack of (state, path) tuples
        frontier = [(start_state, [])]
        
        # Explored set to avoid cycles
        explored = set()
        
        # Search loop
        while frontier:
            # Update statistics
            self.max_frontier_size = max(self.max_frontier_size, len(frontier))
            
            # Get next state (LIFO)
            state, path = frontier.pop()
            
            # Skip if already explored
            if state in explored:
                continue
            
            # Mark as explored
            explored.add(state)
            self.nodes_expanded += 1
            self.exploration_order.append(state)
            
            # Check if goal
            if problem.is_goal_state(state):
                return path
            
            # Expand node (reverse order to maintain left-to-right exploration)
            successors = problem.get_successors(state)
            for successor, action, cost in reversed(successors):
                if successor not in explored:
                    frontier.append((successor, path + [action]))
        
        # No solution found
        return None
    
    def get_statistics(self):
        """Return search statistics"""
        return {
            'nodes_expanded': self.nodes_expanded,
            'max_frontier_size': self.max_frontier_size,
            'states_explored': len(self.exploration_order)
        }

## 4. Visualization Functions

In [ ]:
def visualize_search(problem, algorithm, title="Search Algorithm"):
    """Visualize the search process step by step"""
    
    # Run the algorithm
    solution = algorithm.search(problem)
    
    # Create visualization
    grid = problem.grid
    rows, cols = len(grid), len(grid[0])
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Left plot: Final result
    display1 = np.ones((rows, cols, 3))
    
    # Mark obstacles
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 1:
                display1[i, j] = [0, 0, 0]
    
    # Mark explored states
    for state in algorithm.exploration_order:
        if state != problem.start and state != problem.goal:
            display1[state[0], state[1]] = [0.9, 0.9, 1]
    
    # Mark solution path
    if solution:
        current = problem.start
        display1[current[0], current[1]] = [0, 1, 0]
        
        for action in solution:
            # Determine next position based on action
            if action == 'UP': current = (current[0]-1, current[1])
            elif action == 'DOWN': current = (current[0]+1, current[1])
            elif action == 'LEFT': current = (current[0], current[1]-1)
            elif action == 'RIGHT': current = (current[0], current[1]+1)
            
            if current != problem.goal:
                display1[current[0], current[1]] = [0.5, 1, 0.5]
    
    # Mark start and goal
    display1[problem.start[0], problem.start[1]] = [0, 1, 0]
    display1[problem.goal[0], problem.goal[1]] = [1, 0, 0]
    
    ax1.imshow(display1)
    ax1.set_title(f"{title} - Final Result")
    ax1.axis('off')
    
    # Right plot: Exploration order
    display2 = np.ones((rows, cols, 3))
    
    # Mark obstacles
    for i in range(rows):
        for j in range(cols):
            if grid[i][j] == 1:
                display2[i, j] = [0, 0, 0]
    
    # Show exploration order with numbers
    for idx, state in enumerate(algorithm.exploration_order[:20]):  # Show first 20
        ax2.text(state[1], state[0], str(idx+1), ha='center', va='center', fontsize=8)
        if state != problem.start and state != problem.goal:
            display2[state[0], state[1]] = [0.9 - idx*0.03, 0.9 - idx*0.03, 1]
    
    display2[problem.start[0], problem.start[1]] = [0, 1, 0]
    display2[problem.goal[0], problem.goal[1]] = [1, 0, 0]
    
    ax2.imshow(display2)
    ax2.set_title("Exploration Order (numbers show sequence)")
    ax2.axis('off')
    
    # Add statistics
    stats = algorithm.get_statistics()
    stats_text = f"Nodes expanded: {stats['nodes_expanded']}\n"
    stats_text += f"Max frontier: {stats['max_frontier_size']}\n"
    if solution:
        stats_text += f"Solution length: {len(solution)}"
    else:
        stats_text += "No solution found"
    
    fig.text(0.5, 0.02, stats_text, ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    return solution

## 5. Test Both Algorithms

In [ ]:
# Create a test problem
test_grid = [
    [0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 1, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0],
]

problem = GridSearchProblem(test_grid, (0, 0), (4, 5))

# Test BFS
print("=" * 50)
print("BREADTH-FIRST SEARCH")
print("=" * 50)
bfs = BreadthFirstSearch()
bfs_solution = visualize_search(problem, bfs, "BFS")
print(f"\nBFS Solution: {bfs_solution}")

In [ ]:
# Test DFS
print("=" * 50)
print("DEPTH-FIRST SEARCH")
print("=" * 50)
dfs = DepthFirstSearch()
dfs_solution = visualize_search(problem, dfs, "DFS")
print(f"\nDFS Solution: {dfs_solution}")

## 6. Comparison: BFS vs DFS

In [ ]:
def compare_algorithms(problem):
    """Compare BFS and DFS on the same problem"""
    
    # Run both algorithms
    bfs = BreadthFirstSearch()
    dfs = DepthFirstSearch()
    
    bfs_solution = bfs.search(problem)
    dfs_solution = dfs.search(problem)
    
    # Create comparison table
    comparison = {
        'Algorithm': ['BFS', 'DFS'],
        'Solution Found': [bfs_solution is not None, dfs_solution is not None],
        'Solution Length': [
            len(bfs_solution) if bfs_solution else 'N/A',
            len(dfs_solution) if dfs_solution else 'N/A'
        ],
        'Nodes Expanded': [bfs.nodes_expanded, dfs.nodes_expanded],
        'Max Frontier': [bfs.max_frontier_size, dfs.max_frontier_size]
    }
    
    # Display as table
    import pandas as pd
    df = pd.DataFrame(comparison)
    print(df.to_string(index=False))
    
    # Visualize exploration patterns
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    
    for ax, alg, name in [(ax1, bfs, 'BFS'), (ax2, dfs, 'DFS')]:
        display = np.ones((problem.rows, problem.cols))
        
        # Mark obstacles
        for i in range(problem.rows):
            for j in range(problem.cols):
                if problem.grid[i][j] == 1:
                    display[i, j] = 0
        
        # Heat map of exploration order
        max_order = len(alg.exploration_order)
        for idx, state in enumerate(alg.exploration_order):
            if display[state[0], state[1]] == 1:  # Not an obstacle
                display[state[0], state[1]] = 0.3 + (0.7 * idx / max_order)
        
        im = ax.imshow(display, cmap='RdYlBu_r', vmin=0, vmax=1)
        ax.set_title(f"{name} Exploration Heat Map")
        ax.axis('off')
    
    plt.colorbar(im, ax=[ax1, ax2], label='Exploration Order (early to late)')
    plt.tight_layout()
    plt.show()

# Run comparison
print("\n" + "=" * 50)
print("ALGORITHM COMPARISON")
print("=" * 50 + "\n")
compare_algorithms(problem)

## 7. When to Use Which Algorithm?

### Use BFS when:
- You need the **shortest path** (for uniform costs)
- The solution is **not deep** in the tree
- You can afford the **memory** overhead
- **Completeness** is important

### Use DFS when:
- **Memory is limited**
- Solutions are **deep** in the tree
- You don't need the optimal solution
- The search space is tree-like (no cycles)

## 8. Advanced: Bidirectional Search

Search from both start and goal simultaneously:

In [ ]:
class BidirectionalSearch:
    """Bidirectional BFS implementation"""
    
    def __init__(self):
        self.nodes_expanded = 0
        self.forward_explored = set()
        self.backward_explored = set()
    
    def search(self, problem: SearchProblem) -> List[str]:
        """
        Search from both start and goal
        """
        start = problem.get_start_state()
        goal = problem.goal  # Assumes we know the goal state
        
        if start == goal:
            return []
        
        # Two frontiers
        forward_frontier = deque([(start, [])])
        backward_frontier = deque([(goal, [])])
        
        # Two explored sets with path information
        forward_paths = {start: []}
        backward_paths = {goal: []}
        
        while forward_frontier or backward_frontier:
            # Expand forward frontier
            if forward_frontier:
                state, path = forward_frontier.popleft()
                
                if state in backward_paths:
                    # Found connection!
                    return path + self._reverse_path(backward_paths[state])
                
                if state not in self.forward_explored:
                    self.forward_explored.add(state)
                    self.nodes_expanded += 1
                    
                    for successor, action, _ in problem.get_successors(state):
                        if successor not in self.forward_explored:
                            new_path = path + [action]
                            forward_frontier.append((successor, new_path))
                            if successor not in forward_paths:
                                forward_paths[successor] = new_path
            
            # Expand backward frontier
            if backward_frontier:
                state, path = backward_frontier.popleft()
                
                if state in forward_paths:
                    # Found connection!
                    return forward_paths[state] + self._reverse_path(path)
                
                if state not in self.backward_explored:
                    self.backward_explored.add(state)
                    self.nodes_expanded += 1
                    
                    for successor, action, _ in problem.get_successors(state):
                        if successor not in self.backward_explored:
                            new_path = [action] + path
                            backward_frontier.append((successor, new_path))
                            if successor not in backward_paths:
                                backward_paths[successor] = new_path
        
        return None
    
    def _reverse_path(self, path):
        """Reverse actions for backward path"""
        reverse_map = {'UP': 'DOWN', 'DOWN': 'UP', 'LEFT': 'RIGHT', 'RIGHT': 'LEFT'}
        return [reverse_map.get(action, action) for action in reversed(path)]

# Test bidirectional search
print("\n" + "=" * 50)
print("BIDIRECTIONAL SEARCH")
print("=" * 50)

bi_search = BidirectionalSearch()
bi_solution = bi_search.search(problem)
print(f"Solution: {bi_solution}")
print(f"Nodes expanded: {bi_search.nodes_expanded}")
print(f"Forward explored: {len(bi_search.forward_explored)}")
print(f"Backward explored: {len(bi_search.backward_explored)}")

## 9. Practice Exercises

### Exercise 1: Iterative Deepening DFS
Implement IDDFS that combines benefits of BFS (completeness, optimality) with DFS space efficiency:

In [ ]:
class IterativeDeepeningDFS:
    """
    TODO: Implement IDDFS
    - Run DFS with increasing depth limits
    - Start with depth 0, then 1, 2, ...
    - Stop when solution found
    """
    
    def __init__(self, max_depth=50):
        self.max_depth = max_depth
        self.nodes_expanded = 0
    
    def search(self, problem: SearchProblem) -> List[str]:
        # TODO: Implement iterative deepening
        pass
    
    def depth_limited_search(self, problem, limit):
        # TODO: Implement DFS with depth limit
        pass

# Test your implementation
# iddfs = IterativeDeepeningDFS()
# solution = iddfs.search(problem)
# print(f"IDDFS Solution: {solution}")

### Exercise 2: Graph Search vs Tree Search
Modify algorithms to work without cycle checking (tree search):

In [ ]:
class TreeSearchBFS:
    """
    TODO: Implement BFS without explored set
    - What happens with cycles?
    - When might this be useful?
    """
    
    def search(self, problem: SearchProblem) -> List[str]:
        # TODO: Implement tree search version
        pass

# Compare tree search vs graph search
# - Create a problem with cycles
# - Run both versions
# - Compare nodes expanded

### Exercise 3: Path Cost Tracking
Modify BFS/DFS to track and return path costs:

In [ ]:
class CostTrackingBFS:
    """
    TODO: Modify BFS to track path costs
    - Return both solution and total cost
    - Handle non-uniform costs
    """
    
    def search_with_cost(self, problem: SearchProblem) -> Tuple[List[str], float]:
        # TODO: Return (actions, total_cost)
        pass

## 10. Key Takeaways

### BFS:
✅ **Pros**: Complete, optimal (uniform costs), systematic exploration  
❌ **Cons**: High memory usage O(b^d), can be slow for deep solutions

### DFS:
✅ **Pros**: Low memory usage O(bm), fast for deep solutions  
❌ **Cons**: Not complete, not optimal, can get stuck

### Important Concepts:
1. **Frontier Management**: Queue (BFS) vs Stack (DFS)
2. **Cycle Detection**: Essential for graph search
3. **Completeness vs Efficiency**: Trade-offs in algorithm choice
4. **Exploration Strategy**: How it affects solution quality

## Next Steps
Next notebook: Uniform Cost Search - Finding optimal paths with variable costs!